<a href="https://colab.research.google.com/github/Blvisse/DataMunging/blob/main/DataMunging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade neptune-client

In [12]:
import neptune.new as neptune

In [13]:
run = neptune.init(project='blaisepke/data-munging',
                   api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlYmViMmY2ZS1jNWFjLTRlMWMtYTdlOS00OGMxOWE0YTAzZDIifQ==') # your credentials


Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/blaisepke/data-munging/e/DAT-5
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [ ]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"blaisepapa","key":"ee837bf3e6cabb098df27ae9af567c0d"}'}

In [4]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [5]:
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

ecommerce-behavior-data-from-multi-category-store.zip: Skipping, found more recently modified local copy (use --force to force download)


In [13]:
!pip install dask[dataframe]

     |████████████████████████████████| 118 kB 4.1 MB/s 


In [3]:
import dask
import dask.dataframe as dd

In [4]:
# import shutil
# shutil.unpack_archive('/content/ecommerce-behavior-data-from-multi-category-store.zip' )


import zipfile
with zipfile.ZipFile('/content/ecommerce-behavior-data-from-multi-category-store.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

In [5]:
%%time
dataDask = dd.read_csv('/content/data/2019-Nov.csv')

CPU times: user 87 ms, sys: 7.6 ms, total: 94.6 ms
Wall time: 194 ms


In [2]:
%%time
import pandas as pd


dataPandas=pd.read_csv('/content/data/2019-Nov.csv')

CPU times: user 1min 44s, sys: 22.6 s, total: 2min 6s
Wall time: 3min 11s


Huge notable difference between the processing of dask and pandas 

Dask takes 78.8ms with pandas taking 3min 16 seconds to process the same dataset

In [14]:
dataDask

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
npartitions=141,,,,,,,,,
,object,object,int64,int64,object,object,float64,int64,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [17]:
%%time

dataPandas

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
...,...,...,...,...,...,...,...,...,...
67501974,2019-11-30 23:59:58 UTC,view,15700137,2053013559733912211,NaN,NaN,277.74,532714000,02b4131c-0112-4231-aafa-ceaa08e77c1b
67501975,2019-11-30 23:59:58 UTC,view,28719425,2053013565639492569,apparel.shoes,baden,62.81,545223467,734c5eef-0742-4f8b-9d22-48f75b0bc359
67501976,2019-11-30 23:59:59 UTC,view,1004833,2053013555631882655,electronics.smartphone,samsung,167.03,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb
67501977,2019-11-30 23:59:59 UTC,view,2701706,2053013563911439225,appliances.kitchen.refrigerators,samsung,566.27,531607492,368ddc8b-5db9-40fb-b7ff-b6582a1192c0


Dask unlike pandas when called doesn't display a section of the dataset bu instead an object, this is due to the lazy nature of dask and it only carries out operations when neccessary 

We use the head function instead to see a portion of data 

In [18]:
%%time 

dataDask.head()

CPU times: user 816 ms, sys: 154 ms, total: 970 ms
Wall time: 970 ms


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


## Data Operations

In [11]:
%%time
import time
import resource 

time_start = time.process_time()
#run your code


print(dataDask.columns)

time_elapsed = (time.process_time() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print(" \n Time elapsed",time_elapsed)
print("\n MB of memory used",memMb)




CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 11.9 µs


In [35]:
%%time 
time_start = time.process_time()
#run your code


print(dataPandas.columns)
run['sys/tags'].add('Python')
run['Pandas Cols']=dataPandas.columns

time_elapsed = (time.process_time() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print(" \n Time elapsed",time_elapsed)
print("\n MB of memory used",memMb)
run['Time'].log(time_elapsed)
run['resource'].log(memMb)

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')
 
 Time elapsed 0.005164522000029592

 MB of memory used 11.624961853027344
CPU times: user 8.81 ms, sys: 1.16 ms, total: 9.97 ms
Wall time: 12 ms


We shall test on a simple groupby funtion

In [37]:
eventPrice=dataDask.groupby('event_type').agg({'price':sum})
eventPrice

,price
npartitions=1,
,float64
,...


In [40]:
eventPandas=dataPandas.groupby('event_type').agg({'price':sum})
eventPandas

,price
event_type,
cart,8.783092e+08
purchase,2.751949e+08
view,1.858808e+10


In [39]:
time_start = time.process_time()
#run your code


print(eventPrice.compute())
run['sys/tags'].add('Python-Dask')
run['Dask GroupBy']=eventPrice.compute()

time_elapsed = (time.process_time() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print(" \n Time elapsed",time_elapsed)
print("\n MB of memory used",memMb)
run['Time'].log(time_elapsed)
run['resource'].log(memMb)

                   price
event_type              
cart        8.783092e+08
purchase    2.751949e+08
view        1.858808e+10
 
 Time elapsed 265.83995043400006

 MB of memory used 11.624961853027344


In [42]:
time_start = time.process_time()
#run your code


print(eventPandas)
run['sys/tags'].add('Python')
run['Python Event']=eventPandas

time_elapsed = (time.process_time() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print(" \n Time elapsed",time_elapsed)
print("\n MB of memory used",memMb)
run['Time'].log(time_elapsed)
run['resource'].log(memMb)

                   price
event_type              
cart        8.783092e+08
purchase    2.751949e+08
view        1.858808e+10
 
 Time elapsed 0.02009888800000681

 MB of memory used 11.624961853027344


###Accesing a record 

In [ ]:
t=dataDask['event_type'].compute()

In [60]:
t

Dask Series Structure:
npartitions=141
    object
       ...
     ...  
       ...
       ...
Name: event_type, dtype: object
Dask Name: getitem, 564 tasks

In [14]:
time_start = time.process_time()
#run your code


print(dataPandas['event_type'])
run['sys/tags'].add('Python Pandas')
run['Python Column']=dataPandas['event_type']

time_elapsed = (time.process_time() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print(" \n Time elapsed",time_elapsed)
print("\n MB of memory used",memMb)
run['Time'].log(time_elapsed)
run['resource'].log(memMb)



# dataPandas['event_type']

0           view
1           view
2           view
3           view
4           view
            ... 
67501974    view
67501975    view
67501976    view
67501977    view
67501978    view
Name: event_type, Length: 67501979, dtype: object
 
 Time elapsed 0.009455419999994774

 MB of memory used 11.473270416259766


### Calculating null values and remving them

In [19]:
time_start = time.process_time()
#run your code


print(dataPandas.isnull().sum())
run['sys/tags'].add('Python Pandas')
run['Python Nulls']=dataPandas.isnull().sum()

time_elapsed = (time.process_time() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print(" \n Time elapsed",time_elapsed)
print("\n MB of memory used",memMb)
run['Time'].log(time_elapsed)
run['resource'].log(memMb)



# dataPandas['event_type']




event_time              0
event_type              0
product_id              0
category_id             0
category_code    21898171
brand             9224078
price                   0
user_id                 0
user_session           10
dtype: int64
 
 Time elapsed 31.01279688599999

 MB of memory used 11.473270416259766


In [20]:
time_start = time.process_time()
#run your code


print(dataDask.isnull().sum().compute())
run['sys/tags'].add('Python Dusk')
run['Dusk Nulls']=dataDask.isnull().sum().compute()

time_elapsed = (time.process_time() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print(" \n Time elapsed",time_elapsed)
print("\n MB of memory used",memMb)
run['Time'].log(time_elapsed)
run['resource'].log(memMb)






event_time              0
event_type              0
product_id              0
category_id             0
category_code    21898171
brand             9224078
price                   0
user_id                 0
user_session           10
dtype: int64
 
 Time elapsed 287.803047274

 MB of memory used 11.473270416259766


In [22]:
time_start = time.process_time()
#run your code



run['sys/tags'].add('Python Dusk')

dataDask.dropna()
time_elapsed = (time.process_time() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print(" \n Time elapsed",time_elapsed)
print("\n MB of memory used",memMb)
run['Time'].log(time_elapsed)
run['resource'].log(memMb)






 
 Time elapsed 0.012011850999897433

 MB of memory used 11.473270416259766


In [24]:
time_start = time.process_time()
#run your code



run['sys/tags'].add('Python Pandas')

dataPandas.dropna()
time_elapsed = (time.process_time() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print(" \n Time elapsed",time_elapsed)
print("\n MB of memory used",memMb)
run['Time'].log(time_elapsed)
run['resource'].log(memMb)






 
 Time elapsed 24.964063213999907

 MB of memory used 11.50042724609375


In [33]:
!pip install -U PyYAML

     |████████████████████████████████| 636 kB 3.9 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
import yaml

with open('pandas.yaml', 'w') as file:
  documents = yaml.dump(dataPandas.columns, file)

In [8]:
with open('pandas.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    document = yaml.load(file, Loader=yaml.FullLoader)
    print(document)



# doc = yaml.load('pandas.yaml',Loader=yaml.FullLoader)
# print(doc)

ConstructorError: ignored

In [16]:
dataPandas.columns

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [23]:

%%writefile datafile.yaml
file_type: csv
dataset_name: ecommerce-behavior-data
file_name: 2019-Nov

inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - event_type
    - event_time
    - product_type
    - product_id
    - category_id
    - category_code
    - brand
    - price
    - user_id
    - user_sesion


Overwriting datafile.yaml


In [24]:
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
          pass
            


# Read config file
config_data =read_config_file("datafile.yaml")

In [26]:
config_data

{'columns': ['event_type',
  'event_time',
  'product_type',
  'product_id',
  'category_id',
  'category_code',
  'brand',
  'price',
  'user_id',
  'user_sesion'],
 'dataset_name': 'ecommerce-behavior-data',
 'file_name': '2019-Nov',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1}

In [27]:
def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [ ]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./data/" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

In [ ]:
col_header_val(df,config_data)